### 1NN против RandomForest
В этом задании будет использоваться датасет ```digits``` из ```sklearn.datasets```. Оставьте последние 25% объектов для контроля качества, разделив x и y на ```x_train, y_train и x_test, y_test```.

Целью задания будет реализовать самый простой метрический классификатор — метод ближайшего соседа, а также сравнить качество работы реализованного вами 1NN с ```RandomForestClassifier``` из sklearn на 1000 деревьях.

**1)** Реализуйте самостоятельно метод одного ближайшего соседа с евклидовой метрикой для задачи классификации. Можно не извлекать корень из суммы квадратов отклонений, т.к. корень — монотонное преобразование и не влияет на результат работы алгоритма.

Никакой дополнительной работы с признаками в этом задании делать не нужно — мы еще успеем этим заняться в других курсах. Ваша реализация может быть устроена следующим образом: можно для каждого классифицируемого объекта составлять список пар (расстояние до точки из обучающей выборки, метка класса в этой точке), затем сортировать этот список (по умолчанию сортировка будет сначала по первому элементу пары, затем по второму), а затем брать первый элемент (с наименьшим расстоянием).

Сортировка массива длиной N требует порядка N log N сравнений (строже говоря, она работает за O(N log N)). Подумайте, как можно легко улучшить получившееся время работы. Кроме простого способа найти ближайший объект всего за N сравнений, можно попробовать придумать, как разбить пространство признаков на части и сделать структуру данных, которая позволит быстро искать соседей каждой точки. За выбор метода поиска ближайших соседей в KNeighborsClassifier из sklearn отвечает параметр algorithm — если у вас уже есть некоторый бэкграунд в алгоритмах и структурах данных, вам может быть интересно познакомиться со структурами данных ball tree и kd tree.

Доля ошибок, допускаемых 1NN на тестовой выборке, — ответ в задании 1.

In [96]:
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

import numpy as np

### Загрузка данных

In [89]:
digits = load_digits()

# На проверку оставляем 25% последних объектов, т.е. 450 объектов
x_train, y_train, x_test, y_test = digits.data[:-450], digits.target[:-450], digits.data[-450:], digits.target[-450:]

### 1 NN Implementation

In [93]:
def find_nearest_neighbor(data_point, x, y):
    n_objects = x.shape[0]
    distances = {}
    for indx in range(n_objects):
        distances[indx] = np.sum((data_point - x[indx])**2)
        
    min_distance = sorted(distances.values())[0]
    min_indx_obj = list(distances.values()).index(min_distance)
    pred_label = y[min_indx_obj]
    return pred_label

def calculate_accuracy(x,y):
    pred_labels = []
    for indx in range(x.shape[0]):
        pred_labels.append(find_nearest_neighbor(data_point=x[indx], x=x_train, y=y_train))
    
    return np.sum(pred_labels == y)/y.shape[0]

def write_answ(file_name, value):
    with open(file_name, 'w') as f:
        f.write(str(value))

In [91]:
accuracy_test = calculate_accuracy(x_test, y_test)
print('Test Accuracy: ', accuracy_test)
print('Test Error: ', 1-accuracy_test)

Test Accuracy:  0.9622222222222222
Test Error:  0.0377777777777778


In [94]:
write_answ('answ_1.txt', 1-accuracy_test)

### Random Forest
**2)** Теперь обучите на обучающей выборке ```RandomForestClassifier(n_estimators=1000)``` из ```sklearn```. Сделайте прогнозы на тестовой выборке и оцените долю ошибок классификации на ней. Эта доля — ответ в задании 2. 

Обратите внимание на то, как соотносится качество работы случайного леса с качеством работы, пожалуй, одного из самых простых методов — 1NN. Такое различие — особенность данного датасета, но нужно всегда помнить, что такая ситуация тоже может иметь место, и не забывать про простые методы.

In [100]:
# Обучаем лес
rm_forest = RandomForestClassifier(n_estimators=1000)
rm_forest.fit(x_train, y_train)

preds = rm_forest.predict(x_test)
accuracy = accuracy_score(y_test, preds)

print('Test Accuracy: ', accuracy)
print('Test Error: ', 1-accuracy)

Test Accuracy:  0.9355555555555556
Test Error:  0.06444444444444442


In [101]:
write_answ('answ_2.txt', 1-accuracy)